In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
#import requests
import urllib
from datetime import datetime
from noticeEmail import noticeEMail
from selenium.common.exceptions import TimeoutException
import MDAnalysis as mda
import os
import errno
import numpy as np
starttime = datetime.now()


In [2]:
#variables to change
#['name', 'upper leaflet ratio', 'lower leaflet ratio', 'surface area']
waterHeight = "35"
xYLength = "90"
SA = '20'
fattyAcids = [
    #['pal', 4, 4, 19],
    ['palp', "4", "4", SA],
    ['laup', "1", "1", SA],
    #['lau', 1, 1, "19"]
    #['ste', 0, 0, 0],
    ['step', "3", "3", SA],
    #['myr', 0, 0, 0],
    ['myrp', "2", "2", SA]
]
saltConc = '0.4'
saltName = 'NaCl'
temperature = '298.15'
#these must be correct AND they must exist or script won't work!
fileDirectory = '/gpfs/amarolab/monolayers/Boxes/MIX1_NaCl0.4/SA'+SA+'/'
saveFile = fileDirectory+'charmm-gui.tgz'
runFile = fileDirectory+'charmm-gui/namd/run.sh'
#simulator = 'gmx_checked'
#SIMULATOR OPTIONS
#amber = 'amb_checked'
simulator = 'namd_checked'
#gromacs = 'gmx_checked'
#must be gmail
email = 'acdommer@gmail.com'

In [3]:
def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
make_sure_path_exists(fileDirectory)

In [4]:
try:    
    starttime = datetime.now()
    webpage = "http://charmm-gui.org/?doc=input/monolayer_only&step=1"
    driver = webdriver.PhantomJS('phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
    driver.get(webpage)
    wait10 = WebDriverWait(driver, 10)
    wait600 = WebDriverWait(driver, 1000)
    water_thickness = driver.find_element_by_name("hetero_wdist")
    water_thickness.clear()
    water_thickness.send_keys(waterHeight)
    XY_length = driver.find_element_by_name("hetero_lx")
    XY_length.send_keys(xYLength)
    #click on triangles
    for i in range(1,14):
        driver.find_element(By.XPATH, '//img[@src="/images/basic/tri.png"]').click()
    #setting up an array of input Variables
    #[fatty acid name accepted by charmmgui, upper ratio, lower ratio, surface area per lipid]

    for i in fattyAcids:
        driver.find_element_by_name("lipid_ratio[upper]["+i[0]+"]").clear()
        driver.find_element_by_name("lipid_ratio[lower]["+i[0]+"]").clear()
        driver.find_element_by_name("lipid_ratio[area]["+i[0]+"]").clear()
        driver.find_element_by_name("lipid_ratio[upper]["+i[0]+"]").send_keys(i[1]) #upper
        driver.find_element_by_name("lipid_ratio[lower]["+i[0]+"]").send_keys(i[2]) #lower
        driver.find_element_by_name("lipid_ratio[area]["+i[0]+"]").send_keys(i[3]) #surface area
    #click show system info button
    driver.find_element_by_id('hetero_size_button').click()
    hetero_box = driver.find_element_by_id('hetero_size')
    wait10.until(EC.text_to_be_present_in_element((By.ID, 'hetero_size'), 'Calculated Number of Lipids'))
    driver.save_screenshot('screenie.png')
    #next page
    driver.find_element_by_id('nextBtn').click()
    #wait until page loads
    #after ten minutes, time out
    wait600.until(EC.url_contains('step=2'))
    #fill in salt concentration
    driver.find_element_by_id('ion_conc').clear()
    driver.find_element_by_id('ion_conc').send_keys(saltConc)
    salt_name = Select(driver.find_element_by_id('ion_type'))
    salt_name.select_by_visible_text(saltName)
    calculate_ions = driver.find_element_by_xpath('//input[@value="Calculate number of ions"]')
    calculate_ions.click()
    if saltConc == '0.0':
        driver.find_element_by_xpath('//input[@name="ion_checked"]').click()
    driver.find_element_by_id("nextBtn").click()
    #wait until page loads
    wait600.until(EC.url_contains('step=3'))
    driver.find_element_by_id("nextBtn").click()
    wait600.until(EC.text_to_be_present_in_element((By.ID, "fmonolayer_only"), "Assemble Generated Components"))
    driver.find_element_by_id("nextBtn").click()
    wait600.until(EC.url_contains('step=4'))
    driver.find_element_by_name(simulator).click()
    driver.find_element_by_name('temperature').clear()
    driver.find_element_by_name('temperature').send_keys(temperature)
    driver.find_element_by_id('nextBtn').click()
    wait600.until(EC.url_contains('step=5'))
    download_link_element = driver.find_element_by_css_selector('div#membrane_component a.download')
    #driver.save_screenshot('screenie.png')
    download_link = download_link_element.get_attribute('href')
    #print(download_link.get_attribute('href'))
    urllib.urlretrieve(download_link, saveFile)
except TimeoutException:
    runtime = datetime.now() - starttime
    subject = "CharmmGUI Error Notice (saddddd)"
    msg = "Mr. poopy butthole says your file "+fileDirectory+" has timed out in"+str(runtime)+"! You should probably go fix that"
    noticeEMail(starttime, email, msg, subject)

In [5]:
with open('untar.sh', 'a') as file:
    file.write('cd '+fileDirectory+'\n')
    file.write('tar zxvf charmm-gui.tgz\n')
! chmod 744 untar.sh
! ./untar.sh
! rm -rf untar.sh

charmm-gui/checkfft.py
charmm-gui/convert_par2namd.py
charmm-gui/crystal_image.str
charmm-gui/lipid_lib.tar.gz
charmm-gui/membrane_lipid_restraint.namd.str
charmm-gui/membrane_lipid_restraint.str
charmm-gui/membrane_lipid_restraint2.namd.str
charmm-gui/membrane_lipid_restraint2.str
charmm-gui/step3_nlipids_lower.prm
charmm-gui/step3_nlipids_upper.prm
charmm-gui/step3_packing.inp
charmm-gui/step3_packing.out
charmm-gui/step3_packing.pdb
charmm-gui/step3_packing_head.crd
charmm-gui/step3_packing_head.pdb
charmm-gui/step3_packing_head.psf
charmm-gui/step3_packing_pol.str
charmm-gui/step3_size.inp
charmm-gui/step3_size.out
charmm-gui/step3_size.str
charmm-gui/step4.2_waterbox.crd
charmm-gui/step4.2_waterbox.inp
charmm-gui/step4.2_waterbox.out
charmm-gui/step4_components.str
charmm-gui/step4_lipid.crd
charmm-gui/step4_lipid.inp
charmm-gui/step4_lipid.out
charmm-gui/step4_lipid.pdb
charmm-gui/step4_lipid.psf
charmm-gui/step4_lipid_lipid.crd
charmm-gui/step5_assembly.crd
charmm-gui/step5_asse

In [6]:
driver.close()
driver.quit()

In [7]:
with open(runFile, 'a') as file:
    line = 22
    for i in range(0,len(fattyAcids)):
        linestr = str(line)
        file.write('sed -i "'+linestr+'s/.*centers.*/   centers '+waterHeight+'/" membrane_lipid_restraint.namd.col\n')
        line += 20
    for i in range(0, len(fattyAcids)):
        linestr = str(line)
        file.write('sed -i "'+linestr+'s/.*centers.*/   centers -'+waterHeight+'/" membrane_lipid_restraint.namd.col\n')
        line += 20
    file.write('sed -i "27 a outputPressure     1000;                # print pressure tensor every picosecond" *1_equilibration.inp\n')
    file.write('sed -i "25 a outputPressure     1000;                # print pressure tensor every picosecond" *2_equilibration.inp\n')
    file.write('sed -i "25 a outputPressure     1000;                # print pressure tensor every picosecond" *3_equilibration.inp\n')
    file.write('sed -i "25 a outputPressure     1000;                # print pressure tensor every picosecond" *4_equilibration.inp\n')
    file.write('sed -i "25 a outputPressure     1000;                # print pressure tensor every picosecond" *5_equilibration.inp\n')
    file.write('sed -i "25 a outputPressure     1000;                # print pressure tensor every picosecond" *6_equilibration.inp\n')
    file.write('sed -i "23 a outputPressure     1000;                # print pressure tensor every picosecond" *_production.inp\n')
    file.write('sed -i "s/restartfreq.*/restartfreq     1000;/" *.inp\n')
    file.write('sed -i "s/outputEnergies.*/outputEnergies     1000;/" *.inp\n')
    file.write('sed -i "s/useFlexibleCell.*/useFlexibleCell   no;/" *.inp\n')
    file.write('sed -i "s/useConstantRatio.*/useConstantRatio   no;/" *.inp\n')
    file.write('sed -i "s/langevinPiston .*/langevinPiston   off;/" *.inp\n')
    file.write('sed -i "s/^langevinPistonTarget/#langevinPistonTarget/" *.inp\n')
    file.write('sed -i "s/^langevinPistonPeriod/#langevinPistonPeriod/" *.inp\n')
    file.write('sed -i "s/^langevinPistonDecay/#langevinPistonDecay/" *.inp\n')
    file.write('sed -i "s/^langevinPistonTemp/#langevinPistonTemp/" *.inp\n')
    file.write('sed -i "s/^run 25000/run 500000/" step6.2_equilibration.inp\n')
    file.write('sed -i "s/^run 25000/run 500000/" step6.3_equilibration.inp\n')
    file.write('sed -i "s/^run 100000/run 500000/" step6.4_equilibration.inp\n')
    file.write('sed -i "s/^run 100000/run 500000/" step6.5_equilibration.inp\n')
    file.write('sed -i "s/^run 100000/run 500000/" step6.6_equilibration.inp\n')
    file.write('sed -i "s/^run.*/run 5000000;/" step7.1_production.inp\n')
    file.write('namd2 step6.1_equilibration.inp > step6.1_equilibration.log\n')
    file.write('namd2 step6.2_equilibration.inp > step6.2_equilibration.log\n')
    file.write('namd2 step6.3_equilibration.inp > step6.3_equilibration.log\n')
    file.write('namd2 step6.4_equilibration.inp > step6.4_equilibration.log\n')
    file.write('namd2 step6.5_equilibration.inp > step6.5_equilibration.log\n')
    file.write('namd2 step6.6_equilibration.inp > step6.6_equilibration.log\n')
    file.write('namd2 step7.1_production.inp > step7.1_production.log\n')

In [8]:
u = mda.Universe(fileDirectory+'charmm-gui/step5_assembly.xplor_ext.psf', fileDirectory+'charmm-gui/step5_assembly.namd.pdb')

### Calculating Numbers and Salt Concentration

In [9]:
n_lipids = u.select_atoms('not resname TIP3 and not name SOD and not name CLA').n_residues
n_atoms = u.select_atoms('all').n_atoms
n_waters = u.select_atoms('resname TIP3').n_residues
n_sodium = u.select_atoms('name SOD').n_atoms
n_chloride = u.select_atoms('name CLA').n_atoms
n_ions = 0
if n_sodium <= n_chloride: 
    n_ions = n_chloride - n_sodium
else: n_ions = n_sodium - n_chloride
moles_ions = float(n_ions)/(6.022*(10**23))
a3_water = (float(waterHeight)*float(xYLength)*float(xYLength))
L_water = a3_water/((10**(10))**3)
print(L_water)
molarity_salt = moles_ions/L_water
print(molarity_salt)

1.26e-25
0.0


## "true" pH calculator

In [10]:
#information from Heather Allen's group
##C8 pKa 6.4
##C9 5.8
##C10 7.2
##C12 lauric 7.5
##C14 myristic 8.2
##C16 palmitic 8.7
##C18 stearic 10.2
pKaDict = {'myr':8.2, 'myrp':'myr', 'ste': 10.2, 'step':'ste', 'lau':7.5, 'laup': 'lau', 'pal': 8.7, 'palp':'pal' }
#print(pKaDict)
##using henderson hasselbach equation
##pH = pKa + log(A-/HA)
##loop through the fattyAcid array and build a hash
acidDict = {}
pH_array = []
#setting up an object to keep track of the shit
for fattyAcid in fattyAcids: 
    acidDict[fattyAcid[0]] = fattyAcid[1]
#print(acidDict)
for k in acidDict:
    for i in pH_array:
        if k == pH_array[0] or k == pH_array[2]:
            break
    if type(pKaDict[k]) == float:
        pKa = pKaDict[k]
        acid = k
        base = k+'p'
    else:
        acid = k
        base = pKaDict[k]
        pKa = pKaDict[base]
    #if len(acidDict) == 1:
     #   num_base = 1;
     #   num_acid = u.select_atoms('resname '+acid.upper()).n_residues-1
     #   pH = pKa + np.log(float(num_base)/float(num_acid))
     #   pH_array.append([acid, num_acid, base, num_base, pH])
    #else: 
    num_acid = u.select_atoms('resname '+acid.upper()).n_residues
    num_base = u.select_atoms('resname '+base.upper()).n_residues
    if num_base == 0:
        num_base = 1
        num_acid = num_acid-1
    pH = pKa + np.log(float(num_base)/float(num_acid))
    pH_array.append([acid, num_acid, base, num_base, pH])
print(pH_array)
#print(acid, base, pKa)
#boobs = u.select_atoms('resname LAUP').n_residues
#print(acid)
#print(u.select_atoms(acid.upper()).n_residues)

IndexError: list index out of range

### Adding Basic Data to README file

In [ ]:
README = fileDirectory+'README.html'
with open(README, 'a') as file:
    file.write('<style>table, th, td {border: 1px solid black; border-collapse: collapse; padding: 10px;}</style>'+os.linesep)
    file.write('<h1>'+fileDirectory+'</h1>'+os.linesep)
    file.write('<p><h2>System Stats</h2>')
    file.write('<p><b>Size of System: </b>'+str(n_atoms)+' atoms'+os.linesep)
    file.write('<p><b>Temperature: </b>'+temperature+' K')
    file.write('<p><b>Box Information</b>'+os.linesep)
    file.write('<p><table><tr><td>Z height (&#8491;)</td><td>'+waterHeight+'</td></tr><tr><td>X, Y length (&#8491;)</td><td>'+xYLength+'</td></tr></table>'+os.linesep)
    file.write('<p><b>Salt Concentration: </b>'+str(molarity_salt)+" M"+os.linesep)
    file.write('<p><table><tr><th>Name</th><th>Number</th></tr><tr><td>Water</td><td>'+str(n_waters)+'</td></tr><tr><td>Sodium</td><td>'+str(n_sodium)+'</td></tr><tr><td>Chloride</td><td>'+str(n_chloride)+'</td></tr></table>'+os.linesep)
    file.write('<p><b>Lipid Information</b>'+os.linesep)
    file.write('<p><table><tr><th>Lipid Name</th><th>Upper Leaflet Ratio</th><th>Lower Leaflet Ratio</th><th>Number</th><th>Surface Area per Lipid (&#8491;<sup>2</sup>)</th></tr>'+os.linesep)
    for fattyAcid in fattyAcids:
        number = u.select_atoms('resname '+fattyAcid[0].upper()).n_residues
        file.write('<tr><td>'+fattyAcid[0]+'</td><td>'+str(fattyAcid[1])+'</td><td>'+str(fattyAcid[2])+'</td><td>'+str(number)+'</td><td>'+str(fattyAcid[3])+'</td></tr>'+os.linesep)
    file.write('</table>'+os.linesep)
    file.write('<p><b>Calculated pH</b>'+os.linesep)
    file.write('<p><table>')
    for acid in pH_array:
        file.write('<tr><td>'+str(acid[1])+' '+acid[0]+' + '+str(acid[3])+' '+acid[2]+'</td><td>'+str(acid[4])+'</td></tr>'+os.linesep)
    file.write('</table>')

### Add Minimization and Equilibration Methods to README

In [11]:
with open(README, 'a') as file:
    file.write('<p><h2>Minimization</h2><p>Minimization was completed with the built-in CHARMM-GUI minimization method.')
    file.write('<p><h2>Equilibrations</h2>'+os.linesep)
    file.write('<p><table><tr><th>Step #</th><th>Particle Mesh Ewald (&#8491;)</th><th>Timestep (fs)</th><th>Run time (steps)</th><th>Simulation Length (fs)</th><th>Planar Restraints(kcal/mol)</th><th>Dihedral Restraints (kcal/mol)</th></tr>')
    file.close()
for i in range(1, 7):
    with open(fileDirectory+'charmm-gui/namd/step6.'+str(i)+'_equilibration.inp') as file:
        for line in file:
                if line.startswith('cutoff'):
                    witcutoff = line.split(" ")[14][:-1]
                if line.startswith('timestep'):
                    timestep = line.split(" ")[12][:-1]
                if line.startswith('run'):
                    run = line.split(" ")[1]
                if line.startswith('colvars on'):
                    planar = next(file).split(" ")[5][:-3]
                if line.startswith('extraBonds yes'):
                    dihedral = next(file).split("/")[2]
    file.close()
    with open(README, 'a') as file:
        file.write('<tr><td><u>6.'+str(i)+'</u></td>')
        file.write('<td>'+cutoff+' &#8491;</td>'+os.linesep)
        file.write('<td>'+timestep+'</td>'+os.linesep)
        file.write('<td>'+run+'</td>'+os.linesep)
        file.write('<td>'+ str(float(run)*float(timestep))+'</td>'+os.linesep)
        file.write('<td>'+planar+'</td>')
        file.write('<td>'+dihedral+'</td></tr>')
        file.close()
with open(README, 'a') as file:
    file.write('</table>')
    file.close()

NameError: name 'README' is not defined

In [12]:
runtime = datetime.now() - starttime
subject = "CharmmGUI Completed!"
msg = "Mr. poopy butthole says your file "+fileDirectory+" has completed in"+str(runtime)+"!"
noticeEMail(starttime, email, msg, subject)